In [178]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd

In [179]:
quest_link = 'https://otvet.mail.ru/question/48479555'
inner_html_code = str(urlopen(quest_link).read(),'utf-8') #По факту Арс засунь этот весь код в функцию, и тут ток ссылку менять

In [180]:
inner_soup = BeautifulSoup(inner_html_code, "html.parser")

In [181]:
inner_soup = inner_soup.find('script', {'type':'application/ld+json'})
text = str(inner_soup.get_text()) # found block we convert to string, next step it's just work with string
text

'\n        {"@context":"https://schema.org","@graph":[{"@type":"QAPage","mainEntity":{"@type":"Question","name":"Помогите найти фильм по описанию сюжета!","text":"Давным-давно смотрела фильм, название которого не могу вспомнить. Год выхода тоже не знаю, но думаю, конец 90-х, начало 2000-х. Американский вроде бы. Вкратце сюжет таков. Молодая женщина, у которой есть дочь лет 3-4 в один прекрасный день просыпается и не узнаёт свой дом: не её одежда, не её мебель и т. д. Самое главное, что нет её дочери. Все окружающие говорят ей, что у неё никогда не было детей. Однажды она проходит мимо чужого дома и видит свою дочь, вместе с которой чужая женщина. Она, естественно, пытается дочь забрать, но сама девочка её не узнаёт. Все друзья\\\\знакомые считают, что она сошла с ума. В общем, оказывается, что она каким-то образом попала в параллельную реальность, в которой всё по-другому. Кто-то ей подсказывает, что нужно вспомнить, что она делала в день, перед которым всё изменилось. Она говорит, что

In [182]:
def delit(string):
    string = string.replace(":","")
    string = string.replace("'","")
    string = string.replace('"',"")
    string = string[4:]
    string = string[:-1]
    return string

In [183]:
reviews_answers = list(re.findall(r'text\":\"[\\а-яА-ЯёЁ \d()\,\-!\."a-zA-Z\/]*,',text)) #first element - quest, second - review, third - best answer, another - answers
sort_reviews_answers = list(map(delit, reviews_answers)) #del trash
quest = str(list(re.findall(r'name":\"[а-яА-Я \d()?!]*\"\,\"text\"',text)))
quest = list(re.findall(r'[а-яА-Я]{1,}',quest))
final_quest =' '.join(quest)
review = sort_reviews_answers[0]
best_answer = sort_reviews_answers[1] # в mail вопросах наилучший самый первый
another = sort_reviews_answers[2:]
sort_reviews_answers
#text\":\"[а-яА-Я \d()]*\"
#

['Давным-давно смотрела фильм, название которого не могу вспомнить. Год выхода тоже не знаю, но думаю, конец 90-х, начало 2000-х. Американский вроде бы. Вкратце сюжет таков. Молодая женщина',
 '\\Скольжение/Nightmare Street\\ \\n \\nСкачать)',
 'сюда перейдешь там опишешь и тебе ответят',
 'Этот не знаю но посмотри на подобии \\Остров проклятых\\ фильм очень запутанный с первого раза не поймёшь',
 'Теперь самой стало интерессно посмотреть ...',
 'Не помню название, но этот фильм с Джулианной Мур в главной роли!)))',
 'Помогите найти фильм, две бабушки подруги ухаживают за могилкой неизвестного солдата.',
 'Помогите найти фильм бизнес леди хочет родить для себя ребенка для этого она наняла музыканта тот посилился у ее квартире',
 'Скольжение (1998)',
 'Здравствуйте, помогите найти, пожалуйста, фильм! Смотрела давно где с 2010 года. Запомнился сильно',
 'Здравствуйте всем\\nПомогите найти старенький фильм/сериал руский\\nОна далеко не красавица жила в старенькой квартире, он раздражающий

In [184]:
header = ['Quest','Review','Popular','Another','Questslink']
data = [final_quest, review, best_answer, another, quest_link]
with open('Mailreview.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

In [185]:
file = pd.read_csv('Mailreview.csv')
file

,Quest,Review,Popular,Another,Questslink
0,Помогите найти фильм по описанию сюжета,"Давным-давно смотрела фильм, название которого...",\Скольжение/Nightmare Street\ \n \nСкачать),"['сюда перейдешь там опишешь и тебе ответят', ...",https://otvet.mail.ru/question/48479555
